In [1]:
import yaml
import awkward as aw
import uproot
from util import get_minit_from_procces_file, save_array_to_file, significance, get_cutted_files, ks_weighted
import math
import numpy as np
from IPython.display import Audio
from sklearn.preprocessing import MinMaxScaler
import time
import tensorflow as tf
from sort_jets import *
from tensorflow.keras import layers
import cloudpickle


2022-12-01 21:43:34.290530: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 21:43:35.170181: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-01 21:43:35.310622: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-01 21:43:35.310677: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
with open('config.yaml') as conf_file:
    config = yaml.load(conf_file, Loader=yaml.Loader) 

#### Prepare the cuted files 

In [3]:
do_cut = 0
with_VBF_cut = False
if with_VBF_cut:
    VBF_cut = '_with_VBF_cut'
else:
    VBF_cut = '_without_VBF_cut'


In [4]:
# # Add sorted jets branches
# # Run only once
# for process in config['ttree']['files'].keys():
#     temp_data = get_minit_from_procces_file(process, config, show_stat=False, do_cut=do_cut, with_VBF_cut=with_VBF_cut) #True)
#     print("first part done")
#     temp_data["isVBF"] = 1 if "VBF" in process  else 0
#     temp_data["isGGF"] = 1 if "GGF" in process  else 0
#     # temp_data["weight"] = temp_data["scale1fb"] * temp_data["intLumi"]
#     # Add branches with sorted jets for rnn
#     r = sort_jets(process, config)
#     temp_data["jets_pt_sorted"] =  aw.values_astype(aw.Array(r["jets_pt_sorted"]), "float32") 
#     temp_data["jets_eta_sorted"] = aw.values_astype(aw.Array(r["jets_eta_sorted"]), "float32") 
#     temp_data["jets_phi_sorted"] = aw.values_astype(aw.Array(r["jets_phi_sorted"]), "float32") 
#     temp_data["jets_e_sorted"] = aw.values_astype(aw.Array(r["jets_e_sorted"]), "float32") 
#     new_branch = ["isVBF", "isGGF", "jets_pt_sorted","jets_eta_sorted","jets_phi_sorted","jets_e_sorted"]
#     if "GGF" in process:
#         new_branch_ggf=["jet1_pt", "jet1_eta", "jet1_phi", "jet1_e", 
#                         "jet2_pt", "jet2_eta", "jet2_phi", "jet2_e",
#                         "dphijj", "signetajj", "mjj", "detajj",
#                         "njet30", "scale1fb", "intLumi"]
#         for branch in new_branch_ggf:
#             temp_data[branch] = aw.values_astype(aw.Array(r[branch]), "float32")
#         new_branch+=new_branch_ggf
#     temp_data["weight"] = temp_data["scale1fb"] * temp_data["intLumi"]
#     save_array_to_file(temp_data, f"test/{process}_{do_cut}_cuts{VBF_cut}.root", config, extra_branch=new_branch)
# del temp_data
# del r

#### get cuted files 

In [5]:
# cache = uproot.LRUArrayCache("1 GB")
data_chain = uproot.lazy([f"output/test/{procees}_{do_cut}_cuts{VBF_cut}.root:miniT" for procees in config['ttree']['files'].keys()])#,array_cache=cache)
# consider only events with weight>0
data_s = aw.flatten(data_chain.mask[data_chain["isVBF"] == True].mask[data_chain["weight"]>=0], axis=0)
# use only ggf events with njet30>1 for training and fitting
data_b = aw.flatten(data_chain.mask[data_chain["isVBF"] == False].mask[data_chain["weight"]>=0], axis=0)
# w=data_chain["weight"]
# cache
# data_chain


Check the significance

In [6]:
s = aw.sum(data_s['weight'])
b = aw.sum(data_b['weight'])
s_e = len(data_s["weight"])
b_e = len(data_b)
s_b = s/b
# del data_s2
print(aw.sum(data_s['isVBF']))
print(f"signal: {s}, bkg: {b}, significance: {s/math.sqrt(s+b)}, asimov_significance: {significance(s,b)}")
print(f"No cuts s/b= {s_b}")
print(f"s_mc_events: {s_e}, b_mc_events: {b_e}")
# cache

491044
signal: 10909.44921875, bkg: 6074543.5, significance: 4.422383172057975, asimov_significance: 4.425019382446251
No cuts s/b= 0.0017959290416911244
s_mc_events: 491044, b_mc_events: 4136843


create the split train test

In [7]:
training_frac = 0.7
val_frac = 0.15
array_len = len(data_s["weight"])
mask = np.zeros(array_len, dtype=int)
mask1=mask.copy()
mask2=mask.copy()
mask3 = mask.copy()
mask[:int(array_len * training_frac)] = 1
mask[int(array_len * training_frac):int(array_len * (training_frac+val_frac))] = 2
np.random.shuffle(mask)
for i in range(len(mask)):
    if mask[i]==1:
        mask1[i] = 1
    elif mask[i]==2:
        mask2[i] = 1
    elif mask[i]==0:
        mask3[i] = 1
del mask
mask1 = aw.Array(mask1.astype(bool))
mask2 = aw.Array(mask2.astype(bool))
mask3 = aw.Array(mask3.astype(bool))
s_training = aw.flatten(data_s.mask[mask1], axis=0)
s_val = aw.flatten(data_s.mask[mask2],axis=0)
s_test = aw.flatten(data_s.mask[mask3],axis=0)
del mask1
del mask2
del mask3
# cache


In [8]:
array_len = len(data_b["weight"])
mask = np.zeros(array_len, dtype=int)
mask1=mask.copy()
mask2=mask.copy()
mask3 = mask.copy()
mask[:int(array_len * training_frac)] = 1
mask[int(array_len * training_frac):int(array_len * (training_frac+val_frac))] = 2
np.random.shuffle(mask)
for i in range(len(mask)):
    if mask[i]==1:
        mask1[i] = 1
    elif mask[i]==2:
        mask2[i] = 1
    elif mask[i]==0:
        mask3[i] = 1
del mask
mask1 = aw.Array(mask1.astype(bool))
mask2 = aw.Array(mask2.astype(bool))
mask3 = aw.Array(mask3.astype(bool))
# use only ggf events with njet30>1 for training and fitting
b_training = aw.flatten(data_b.mask[mask1].mask[data_b["njet30"]>1], axis=0)
b_val = aw.flatten(data_b.mask[mask2].mask[data_b["njet30"]>1],axis=0)
b_test = aw.flatten(data_b.mask[mask3].mask[data_b["njet30"]>1],axis=0)
# save events with njets<2 for statistics
b_training_1_0_jets = aw.flatten(data_b.mask[mask1].mask[data_b["njet30"]<=1], axis=0)
b_val_1_0_jets = aw.flatten(data_b.mask[mask2].mask[data_b["njet30"]<=1],axis=0)
b_test_1_0_jets = aw.flatten(data_b.mask[mask3].mask[data_b["njet30"]<=1],axis=0)

del mask1
del mask2
del mask3
# cache

In [9]:
b_training_1_0_jets = aw.to_numpy(b_training_1_0_jets[["njet30", "weight"]])
b_val_1_0_jets = aw.to_numpy(b_val_1_0_jets[["njet30", "weight"]])
b_test_1_0_jets = aw.to_numpy(b_test_1_0_jets[["njet30", "weight"]])

In [10]:
path = "/home/cristobalchavez21/notebooks/production_mode_classifier/dpj/DPJAnalisis/output/train_test/"
np.save(path+"b_training_1_0_jets", b_training_1_0_jets)
np.save(path+"b_val_1_0_jets", b_val_1_0_jets)
np.save(path+"b_test_1_0_jets", b_test_1_0_jets)



#### Prepare Data for training

In [11]:
BDT_inputs = ["jets_pt_sorted", "jets_eta_sorted", "jets_phi_sorted",
                "jets_e_sorted"
             ]
# # cache

In [12]:
s_w_train = aw.to_numpy(s_training["weight"])
s_y_train = aw.to_numpy(s_training["isVBF"])
b_w_train = aw.to_numpy(b_training["weight"])
b_y_train = aw.to_numpy(b_training["isVBF"])

s_w_val = aw.to_numpy(s_val["weight"])
s_y_val = aw.to_numpy(s_val["isVBF"])
b_w_val = aw.to_numpy(b_val["weight"])
b_y_val = aw.to_numpy(b_val["isVBF"])

s_w_test = aw.to_numpy(s_test["weight"])
s_y_test = aw.to_numpy(s_test["isVBF"])
b_w_test = aw.to_numpy(b_test["weight"])
b_y_test = aw.to_numpy(b_test["isVBF"])

#### Normalize Data
Also reshapes for RNN

In [13]:
# maximum number of jets to be used in rnn
max_jets = 6

In [14]:
# Zero and None padding of signal and background (training)
norm_layers = []
s_n = len(s_training)
b_n = len(b_training)
dt = []
for branch in BDT_inputs:
    dt.append((branch, float))
s_x_train_norm = np.zeros((s_n,max_jets), dtype=dt)
b_x_train_norm = np.zeros((b_n,max_jets), dtype=dt)

for branch in BDT_inputs:
# for branch in ["jets_e_sorted"]:
#######################################
###### ZERO PADDING ###################
#######################################
    # Signal
    s_feature = s_training[branch]
    # Zero padding
    # Add zeros to have fix length sequences
    s_padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        s_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float')
    # None padding for normalization
    s_padded_inputs_none = tf.keras.preprocessing.sequence.pad_sequences(
        s_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float', value=None)
    # s_mask = embedding.compute_mask(s_padded_inputs)

    # Background
    b_feature = b_training[branch]
    # Zero padding
    # Add zeros to have fix length sequences
    b_padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        b_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float')
    # None padding for normalization
    b_padded_inputs_none = tf.keras.preprocessing.sequence.pad_sequences(
        b_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float', value=None)
    # b_mask = embedding.compute_mask(b_padded_inputs)

    # Concatenate padded signal and bakcground to normalize considering all events
    s_b_padded = np.concatenate((s_padded_inputs, b_padded_inputs))
    # creating mask to ignore zeros in normalization
    embedding = layers.Embedding(input_dim=5000, output_dim=19, mask_zero=True)
    s_b_mask = embedding.compute_mask(s_b_padded)

#################################################
########## FITTING NORMALIZATION LAYER ##########
#################################################
    # normalization layer
    norm_layer = layers.Normalization(axis=None)
    # fitting normalization layer to signal+bkg ignoring zeros
    norm_layer.adapt(tf.boolean_mask(s_b_padded, s_b_mask))
    # save normalization layer to a list to use on later
    norm_layers.append(norm_layer)

#################################################
######## NORMALIZING ############################
#################################################

    # normalize signal None padded features (normalizing zero padded would change the zeros)
    s_normalized_f = norm_layer(s_padded_inputs_none)
    # change nan back to zeros
    s_normalized_f = np.nan_to_num(s_normalized_f.numpy()) 
    s_x_train_norm[branch] = s_normalized_f

    # normalize background None padded features (normalizing zero padded would change the zeros)
    b_normalized_f = norm_layer(b_padded_inputs_none)
    # change nan back to zeros
    b_normalized_f = np.nan_to_num(b_normalized_f.numpy()) 
    b_x_train_norm[branch] = b_normalized_f
    print(f"{branch} branch done")
s_x_train_norm = s_x_train_norm.view((float, len(BDT_inputs)))
b_x_train_norm = b_x_train_norm.view((float, len(BDT_inputs)))

2022-12-01 21:57:46.336016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-01 21:57:46.336236: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-01 21:57:46.336359: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MSI): /proc/driver/nvidia/version does not exist
2022-12-01 21:57:46.337482: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


jets_pt_sorted branch done
jets_eta_sorted branch done
jets_phi_sorted branch done
jets_e_sorted branch done


In [15]:
# get mean and variance for each feature and put them in a list of dic
mean_var_list = []
for i in range(len(BDT_inputs)):
    # mean and variance are store as layer weights
    layer_weights = norm_layers[i].get_weights()
    dic = {"mean": layer_weights[0], "variance": layer_weights[1], "axis": None}
    mean_var_list.append(dic)
del dic
del layer_weights
mean_var_list

[{'mean': 65538.52, 'variance': 2910983000.0, 'axis': None},
 {'mean': 0.00801053, 'variance': 4.21058, 'axis': None},
 {'mean': 0.008975166, 'variance': 3.280814, 'axis': None},
 {'mean': 356531.2, 'variance': 257906570000.0, 'axis': None}]

In [16]:
# save list of means and variances
path = "/home/cristobalchavez21/notebooks/production_mode_classifier/dpj/DPJAnalisis/output/train_test/"
with open(path+"normalization_params.pkl", "wb") as o_file:
    cloudpickle.dump(mean_var_list, o_file)
# cloudpickle.dump(scaler, open('scaler.pkl', 'wb'))<

In [17]:
# This is how it should be loaded and used
# norm_params = cloudpickle.load(open(f'{path}normalization_params.pkl', 'rb'))
# new_layer = tf.keras.layers.Normalization(**norm_params[0])
# new_layer(np.array([1,20,60000]))


In [18]:
# create array that contains sequence length of each event (number of jets used)
s_seq_len_train = aw.to_numpy(s_training["njet30"])
s_seq_len_train[s_training["njet30"]>max_jets] = max_jets
b_seq_len_train = aw.to_numpy(b_training["njet30"])
b_seq_len_train[b_training["njet30"]>max_jets] = max_jets

In [19]:
from numpy import save
path = "/home/cristobalchavez21/notebooks/production_mode_classifier/dpj/DPJAnalisis/output/train_test/"
save(path+"signal_x_train_norm", s_x_train_norm)
save(path+"bkg_x_train_norm", b_x_train_norm)
save(path+"signal_w_train", s_w_train)
save(path+"bkg_w_train", b_w_train)
save(path+"signal_y_train", s_y_train)
save(path+"bkg_y_train", b_y_train)
save(path+"signal_seq_len_train", s_seq_len_train)
save(path+"bkg_seq_len_train", b_seq_len_train)
del s_training
del b_training

In [20]:
# Zero and None padding of signal and background (validation)
s_n = len(s_val)
b_n = len(b_val)
s_x_val_norm = np.zeros((s_n,max_jets), dtype=dt)
b_x_val_norm = np.zeros((b_n,max_jets), dtype=dt)
index=0
for branch in BDT_inputs:
#######################################
###### ZERO PADDING ###################
#######################################
    # Signal
    s_feature = s_val[branch]
    # Zero padding
    # Add zeros to have fix length sequences
    s_padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        s_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float')
    # None padding for normalization
    s_padded_inputs_none = tf.keras.preprocessing.sequence.pad_sequences(
        s_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float', value=None)
    # s_mask = embedding.compute_mask(s_padded_inputs)

    # Background
    b_feature = b_val[branch]
    # Zero padding
    # Add zeros to have fix length sequences
    b_padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        b_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float')
    # None padding for normalization
    b_padded_inputs_none = tf.keras.preprocessing.sequence.pad_sequences(
        b_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float', value=None)
    # b_mask = embedding.compute_mask(b_padded_inputs)

    # Concatenate padded signal and bakcground to normalize considering all events
    # s_b_padded = np.concatenate((s_padded_inputs, b_padded_inputs))
    # creating mask to ignore zeros in normalization
    # embedding = layers.Embedding(input_dim=5000, output_dim=19, mask_zero=True)
    # s_b_mask = embedding.compute_mask(s_b_padded)

#################################################
########## FITTING NORMALIZATION LAYER ##########
#################################################
    # normalization layer (already fited to trainin data)
    norm_layer = norm_layers[index]

#################################################
######## NORMALIZING ############################
#################################################

    # normalize signal None padded features (normalizing zero padded would change the zeros)
    s_normalized_f = norm_layer(s_padded_inputs_none)
    # change nan back to zeros
    s_normalized_f = np.nan_to_num(s_normalized_f.numpy()) 
    s_x_val_norm[branch] = s_normalized_f

    # normalize background None padded features (normalizing zero padded would change the zeros)
    b_normalized_f = norm_layer(b_padded_inputs_none)
    # change nan back to zeros
    b_normalized_f = np.nan_to_num(b_normalized_f.numpy()) 
    b_x_val_norm[branch] = b_normalized_f
    print(f"{branch} branch done")
    index+=1
s_x_val_norm = s_x_val_norm.view((float, len(BDT_inputs)))
b_x_val_norm = b_x_val_norm.view((float, len(BDT_inputs)))
del b_normalized_f
del b_padded_inputs
del b_padded_inputs_none
del b_feature
del s_feature
del s_normalized_f
del s_padded_inputs
del s_padded_inputs_none

jets_pt_sorted branch done
jets_eta_sorted branch done
jets_phi_sorted branch done
jets_e_sorted branch done


In [21]:
# create array that contains sequence length of each event (number of jets used)
s_seq_len_val = aw.to_numpy(s_val["njet30"])
s_seq_len_val[s_val["njet30"]>max_jets] = max_jets
b_seq_len_val = aw.to_numpy(b_val["njet30"])
b_seq_len_val[b_val["njet30"]>max_jets] = max_jets

In [22]:
save(path+"signal_x_val_norm", s_x_val_norm)
save(path+"bkg_x_val_norm", b_x_val_norm)
save(path+"signal_w_val", s_w_val)
save(path+"bkg_w_val", b_w_val)
save(path+"signal_y_val", s_y_val)
save(path+"bkg_y_val", b_y_val)
save(path+"signal_seq_len_val", s_seq_len_val)
save(path+"bkg_seq_len_val", b_seq_len_val)
del s_val
del b_val

In [23]:
# Zero and None padding of signal and background (test)
s_n = len(s_test)
b_n = len(b_test)
s_x_test_norm = np.zeros((s_n,max_jets), dtype=dt)
b_x_test_norm = np.zeros((b_n,max_jets), dtype=dt)
index=0
for branch in BDT_inputs:
#######################################
###### ZERO PADDING ###################
#######################################
    # Signal
    s_feature = s_test[branch]
    # Zero padding
    # Add zeros to have fix length sequences
    s_padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        s_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float')
    # None padding for normalization
    s_padded_inputs_none = tf.keras.preprocessing.sequence.pad_sequences(
        s_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float', value=None)
    # s_mask = embedding.compute_mask(s_padded_inputs)

    # Background
    b_feature = b_test[branch]
    # Zero padding
    # Add zeros to have fix length sequences
    b_padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        b_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float')
    # None padding for normalization
    b_padded_inputs_none = tf.keras.preprocessing.sequence.pad_sequences(
        b_feature, padding="post", truncating="post", maxlen=max_jets, dtype='float', value=None)
    # b_mask = embedding.compute_mask(b_padded_inputs)

    # Concatenate padded signal and bakcground to normalize considering all events
    # s_b_padded = np.concatenate((s_padded_inputs, b_padded_inputs))
    # creating mask to ignore zeros in normalization
    # embedding = layers.Embedding(input_dim=5000, output_dim=19, mask_zero=True)
    # s_b_mask = embedding.compute_mask(s_b_padded)

#################################################
########## FITTING NORMALIZATION LAYER ##########
#################################################
    # normalization layer (already fited to trainin data)
    norm_layer = norm_layers[index]

#################################################
######## NORMALIZING ############################
#################################################

    # normalize signal None padded features (normalizing zero padded would change the zeros)
    s_normalized_f = norm_layer(s_padded_inputs_none)
    # change nan back to zeros
    s_normalized_f = np.nan_to_num(s_normalized_f.numpy()) 
    s_x_test_norm[branch] = s_normalized_f

    # normalize background None padded features (normalizing zero padded would change the zeros)
    b_normalized_f = norm_layer(b_padded_inputs_none)
    # change nan back to zeros
    b_normalized_f = np.nan_to_num(b_normalized_f.numpy()) 
    b_x_test_norm[branch] = b_normalized_f
    print(f"{branch} branch done")
    index+=1
s_x_test_norm = s_x_test_norm.view((float, len(BDT_inputs)))
b_x_test_norm = b_x_test_norm.view((float, len(BDT_inputs)))
del b_normalized_f
del b_padded_inputs
del b_padded_inputs_none
del b_feature
del s_feature
del s_normalized_f
del s_padded_inputs
del s_padded_inputs_none

jets_pt_sorted branch done
jets_eta_sorted branch done
jets_phi_sorted branch done
jets_e_sorted branch done


In [24]:
# create array that contains sequence length of each event (number of jets used)
s_seq_len_test = aw.to_numpy(s_test["njet30"])
s_seq_len_test[s_test["njet30"]>max_jets] = max_jets
b_seq_len_test = aw.to_numpy(b_test["njet30"])
b_seq_len_test[b_test["njet30"]>max_jets] = max_jets

In [25]:
save(path+"signal_x_test_norm", s_x_test_norm)
save(path+"bkg_x_test_norm", b_x_test_norm)
save(path+"signal_w_test", s_w_test)
save(path+"bkg_w_test", b_w_test)
save(path+"signal_y_test", s_y_test)
save(path+"bkg_y_test", b_y_test)
save(path+"signal_seq_len_test", s_seq_len_test)
save(path+"bkg_seq_len_test", b_seq_len_test)
del s_test
del b_test